**MCW mean: 58, sigma: 5; OW mean: 29; ACC mean: 97**

In [43]:
import pandas as pd
import numpy as np

In [44]:
df = pd.read_csv('THO_Data.csv', usecols=['slidersn', 'JOBNUM', 'WEW_R0_NM', 'OW_R0_DB', 'Pred_ACC_CMR'])
df.shape

(1861552, 5)

In [45]:
df.head()

,slidersn,JOBNUM,WEW_R0_NM,OW_R0_DB,Pred_ACC_CMR
0,545820470B,40097732,55.972,26.476,97.485
1,545820470C,40097732,51.591,23.876,99.703
2,545820470E,40097732,54.254,26.251,99.090
3,545820470F,40097732,53.890,25.313,98.520
4,5458204728,40097732,NaN,NaN,NaN


In [46]:
df.dropna(subset=['WEW_R0_NM', 'OW_R0_DB', 'Pred_ACC_CMR'], how='any', inplace=True)
df.shape

(1779881, 5)

In [47]:
df['slidersn'].nunique()

1779881

In [48]:
pd.set_option('display.float_format', lambda x:'%.3f'%x)
df['WEW_R0_NM'].describe([.01, .05, .95, .99])

count   1779881.000
mean         58.458
std           4.708
min        -100.516
1%           45.758
5%           50.708
50%          58.625
95%          65.647
99%          69.433
max         151.260
Name: WEW_R0_NM, dtype: float64

In [49]:
df = df.loc[df.WEW_R0_NM.between(30,100)]
df.shape

(1779851, 5)

In [50]:
df['WEW_R0_NM'].agg(['count', 'mean', 'std'])

count   1779851.000
mean         58.457
std           4.702
Name: WEW_R0_NM, dtype: float64

In [60]:
def Binning(df = df, n=100, frac=0.9):
    df_copy = df.copy(deep=True)
    MCW_model = np.round(np.random.normal(58, 5.4, len(df)), 3)
    MCW_model.sort()
    MCW_model_cut = pd.cut(MCW_model, n, precision=3, retbins=True)
    MCW_model = pd.DataFrame({'WEW_R0_NM': MCW_model, 'Bin': MCW_model_cut[0]})
    MCW_model_bin = np.round(MCW_model_cut[1],3)
    
    df_copy['WEW_R0_NM'] = df_copy['WEW_R0_NM'].round(decimals=3)
    df_copy.sort_values(by='WEW_R0_NM', ascending=True, inplace=True)
    df_copy_cut = pd.Series(data = pd.cut(df_copy['WEW_R0_NM'], bins=MCW_model_bin), name='Bin')   
    df_copy = pd.concat([df_copy, df_copy_cut], axis=1)
    
    select_size = np.round(MCW_model['Bin'].value_counts()*frac, 0).astype('int64')
    collect_select = pd.DataFrame(dtype='float64')
    for i in select_size.index:
        df_group = df_copy.loc[df_copy['Bin']==i]
        if len(df_group) < select_size[i]:
            df_group = df_group
        else:
            df_group = df_group.sample(n=select_size[i])
        collect_select = pd.concat([collect_select, df_group], axis=0)

    return collect_select

In [73]:
def Find_Match_Bins():
    for i in [0.9, 0.8, 0.7]:
        for j in [10, 50, 100, 200, 300]:
            select_bins = Binning(df, n=j, frac=i)
#             if (select_bins['WEW_R0_NM'].mean() >= 57.97) & (select_bins['WEW_R0_NM'].mean() <= 58.4) & (select_bins['WEW_R0_NM'].std() >= 4.99) & (select_bins['WEW_R0_NM'].mean() <= 5.04):
            if (pd.Series(select_bins['WEW_R0_NM'].mean()).between(57.97, 58.3).values) & (pd.Series(select_bins['WEW_R0_NM'].std()).between(4.99, 5.04).values):
                print('Find it!, Bin:{}, Frac:{}'.format(i, j))
                print(select_bins['WEW_R0_NM'].agg(['count', 'mean', 'std']))
                select_bins[['slidersn']].to_csv('SN_select.csv', index=False)
                return select_bins             
    return select_bins

In [74]:
select_df = Find_Match_Bins()

Find it!, Bin:0.8, Frac:50
count   1336978.000
mean         58.204
std           5.031
Name: WEW_R0_NM, dtype: float64


In [75]:
select_df.head()

,slidersn,JOBNUM,WEW_R0_NM,OW_R0_DB,Pred_ACC_CMR,Bin
647632,5720D2580F,40120478,57.810,29.189,97.654,"(57.048, 58.13]"
407594,570320E11D,40407319,57.160,27.825,96.250,"(57.048, 58.13]"
676085,5721311F26,36097968,58.075,28.489,97.655,"(57.048, 58.13]"
769313,573373DB06,40606088,57.760,27.592,98.081,"(57.048, 58.13]"
1785998,5837008131,39755353,57.710,29.375,97.791,"(57.048, 58.13]"


### Consindering More Paras

In [77]:
select_df.describe()

,JOBNUM,WEW_R0_NM,OW_R0_DB,Pred_ACC_CMR
count,1336978.000,1336978.000,1336978.000,1336978.000
mean,39577585.780,58.204,28.868,96.535
std,1070878.079,5.031,3.401,2.437
min,31008488.000,31.886,2.950,80.835
25%,39097538.000,54.834,26.562,95.447
50%,39744449.000,58.185,28.885,97.230
75%,40412399.000,61.611,31.201,98.229
max,41344965.000,84.875,48.957,106.389


In [79]:
summary_df = select_df.groupby('JOBNUM').agg({'OW_R0_DB': ['count', 'mean'],
                                             'Pred_ACC_CMR': ['mean'],
                                             'WEW_R0_NM':['mean', 'std']})
summary_df.head()

OW_R0_DB        Pred_ACC_CMR WEW_R0_NM      
            count   mean         mean      mean   std
JOBNUM                                               
31008488      325 29.840       96.520    59.353 3.951
31046428      381 26.551       97.138    55.843 4.720
31720175      383 27.234       97.775    55.693 4.836
31905237      372 28.014       97.609    57.116 4.427
32445176      316 29.614       97.002    57.966 4.729

In [80]:
summary_df.columns = ['_'.join(cols).upper()
                     for cols in summary_df.columns.to_flat_index()]
summary_df.head()

,OW_R0_DB_COUNT,OW_R0_DB_MEAN,PRED_ACC_CMR_MEAN,WEW_R0_NM_MEAN,WEW_R0_NM_STD
JOBNUM,,,,,
31008488,325,29.840,96.520,59.353,3.951
31046428,381,26.551,97.138,55.843,4.720
31720175,383,27.234,97.775,55.693,4.836
31905237,372,28.014,97.609,57.116,4.427
32445176,316,29.614,97.002,57.966,4.729


In [82]:
summary_df.rename({'OW_R0_DB_COUNT': 'Qty',
                  'OW_R0_DB_MEAN': 'OW_Mean',
                  'PRED_ACC_CMR_MEAN': 'ACC_Mean',
                  'WEW_R0_NM_MEAN': 'MCW_Mean',
                  'WEW_R0_NM_STD': 'MCW_Std'}, axis=1, inplace=True)
summary_df.head()

,Qty,OW_Mean,ACC_Mean,MCW_Mean,MCW_Std
JOBNUM,,,,,
31008488,325,29.840,96.520,59.353,3.951
31046428,381,26.551,97.138,55.843,4.720
31720175,383,27.234,97.775,55.693,4.836
31905237,372,28.014,97.609,57.116,4.427
32445176,316,29.614,97.002,57.966,4.729


In [86]:
summary_df.to_csv('summary_df.csv')

In [89]:
from sklearn.metrics import DistanceMetric

dist = DistanceMetric.get_metric('euclidean')
dist.pairwise(summary_df[['OW_Mean', 'ACC_Mean']], [29, 97])

ImportError: cannot import name 'DistanceMetric' from 'sklearn.metrics' (C:\Users\1000257489\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\__init__.py)

In [83]:
# def Nor_transform(df, ow_mean=29, acc_mean=97):
#     df_copy = df.copy(deep=True)
#     df_copy['OW_Mean'] = (df_copy['OW_Mean'] - ow_mean)/df_copy['OW_Mean'].std()
#     df_copy['ACC_Mean'] = (df_copy['ACC_Mean'] - acc_mean)/df_copy['ACC_Mean'].std()
    
#     return df_copy

In [84]:
# summary_df_nor = Nor_transform(summary_df)
# summary_df_nor.head()

,Qty,OW_Mean,ACC_Mean,MCW_Mean,MCW_Std
JOBNUM,,,,,
31008488,325,0.476,-0.430,59.353,3.951
31046428,381,-1.388,0.123,55.843,4.720
31720175,383,-1.001,0.694,55.693,4.836
31905237,372,-0.559,0.545,57.116,4.427
32445176,316,0.348,0.001,57.966,4.729
